In [3]:
import tensorflow as tf
import rioxarray as rxr
from tqdm import tqdm 
import xarray as xr 
import numpy as np 
import glob
import os 
import re

In [4]:
%%time 

from functions import Data_nc 

g05=Data_nc( fname="GFS0.5_t2m_heightAboveGround_instant_2022_*.nc", path="../Datos/GFS/2022_std")
g05.path_collection()
g05.loadVar()

era5=Data_nc( fname="era5_2022*.nc", path="../Datos/ERA5/2022_std")
era5.path_collection()
era5.loadVar()

g05.upscaleVar(era5.values_t2m)

CPU times: total: 4.72 s
Wall time: 4.74 s


<xarray.DataArray 't2m' (valid_time: 1464, latitude: 192, longitude: 64)>
dask.array<copy, shape=(1464, 192, 64), dtype=float32, chunksize=(1464, 192, 64), chunktype=numpy.ndarray>
Coordinates:
    heightAboveGround  float64 2.0
  * valid_time         (valid_time) datetime64[ns] 2022-06-21 ... 2022-12-21T...
  * latitude           (latitude) float32 -13.12 -13.38 -13.62 ... -60.62 -60.88
  * longitude          (longitude) float32 -78.88 -78.62 ... -63.38 -63.12
Attributes: (12/29)
    GRIB_paramId:                             167
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      259920
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                2 metre temperature
    GRIB_shortName:                           2t
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            air_temperature

In [5]:
intervalo='2022062100_2022122100'

gfs_parches   = tf.convert_to_tensor(np.load("parches/gfs_"+intervalo+".npy"))
era5_parches  = tf.convert_to_tensor(np.load("parches/era5_"+intervalo+".npy"))
lat_parches   = tf.convert_to_tensor(np.load("parches/lat_"+intervalo+".npy"))
lon_parches   = tf.convert_to_tensor(np.load("parches/lon_"+intervalo+".npy"))
orog_parches  = tf.convert_to_tensor(np.load("parches/orog_"+intervalo+".npy"))
time1_parches = tf.convert_to_tensor(np.load("parches/time1_"+intervalo+".npy"))
time2_parches = tf.convert_to_tensor(np.load("parches/time2_"+intervalo+".npy"))


In [7]:
inp_norm, scale_factor_inp = g05.normalization(gfs_parches)
target_norm, scale_factor_target = era5.normalization(era5_parches)
lat_norm, scale_factor_lat = era5.normalization(lat_parches)
lon_norm, scale_factor_lon = era5.normalization(lon_parches)
orog_norm, scale_factor_orog = era5.normalization(orog_parches)
time_norm, scale_factor_time = era5.normalization(time1_parches)

print(target_norm.shape)

(17544, 3, 32, 32, 1)


#### Divisón de la data

In [95]:
num_registros= era5_parches.shape[0]
idxs_tr= int( num_registros * 0.6 )
idxs_te= round( num_registros * 0.2 ) 
idxs_va= round( num_registros * 0.2 ) 
print(num_registros)
print(idxs_tr)
print(idxs_te)

17544
10526
3509


In [96]:
inp_tr= inp_norm[:idxs_tr, :,:,:,: ]
target_tr= target_norm[: idxs_tr, :,:,:,: ]
lat_tr = lat_norm[: idxs_tr, :,:,:,: ]
lon_tr = lon_norm[: idxs_tr, :,:,:,: ]
orog_tr = orog_norm[:idxs_tr, :,:,:,: ]
time1_tr = time_norm[: idxs_tr, :,:,:,: ]

print(inp_tr.shape)

(10526, 3, 32, 32, 1)


In [98]:
inp_te= inp_norm[ idxs_tr: idxs_tr + idxs_te , :,:,:,: ]
target_te= target_norm[idxs_tr: idxs_tr + idxs_te, :,:,:,: ]
lat_te = lat_norm[idxs_tr: idxs_tr + idxs_te, :,:,:,: ]
lon_te = lon_norm[idxs_tr: idxs_tr + idxs_te, :,:,:,: ]
orog_te = orog_norm[idxs_tr: idxs_tr + idxs_te, :,:,:,: ]
time1_te = time_norm[idxs_tr: idxs_tr + idxs_te, :,:,:,: ]

print(inp_te.shape)

(3509, 3, 32, 32, 1)


In [111]:
inp_va= inp_norm[idxs_tr + idxs_te :, :,:,:,: ]
target_va= target_norm[idxs_tr + idxs_te:, :,:,:,: ]
lat_va = lat_norm[idxs_tr + idxs_te:, :,:,:,: ]
lon_va = lon_norm[idxs_tr + idxs_te:, :,:,:,: ]
orog_va = orog_norm[idxs_tr + idxs_te:, :,:,:,: ]
time1_va = time_norm[idxs_tr + idxs_te:, :,:,:,: ]

print(inp_va.shape)

(3509, 3, 32, 32, 1)


#### Data Shuffle

In [126]:
indices_tr = tf.random.shuffle( tf.range(start=0, limit=tf.shape(inp_tr)[0], dtype=tf.int32), seed=42)
indices_te = tf.random.shuffle( tf.range(start=0, limit=tf.shape(inp_te)[0], dtype=tf.int32), seed=42)
indices_va = tf.random.shuffle( tf.range(start=0, limit=tf.shape(inp_va)[0], dtype=tf.int32), seed=42)

In [127]:
inp_tr=    tf.gather(inp_tr, indices_tr)
target_tr= tf.gather(target_tr, indices_tr)
lat_tr =   tf.gather(lat_tr, indices_tr)
lon_tr =   tf.gather(lon_tr, indices_tr)
orog_tr =  tf.gather(orog_tr, indices_tr)
time1_tr = tf.gather(time1_tr, indices_tr)

In [129]:
inp_te=    tf.gather(inp_te, indices_te)
target_te= tf.gather(target_te, indices_te)
lat_te =   tf.gather(lat_te, indices_te)
lon_te =   tf.gather(lon_te, indices_te)
orog_te =  tf.gather(orog_te, indices_te)
time1_te = tf.gather(time1_te, indices_te)

In [130]:
inp_va=    tf.gather(inp_va, indices_va)
target_va= tf.gather(target_va, indices_va)
lat_va =   tf.gather(lat_va, indices_va)
lon_va =   tf.gather(lon_va, indices_va)
orog_va =  tf.gather(orog_va, indices_va)
time1_va = tf.gather(time1_va, indices_va)

#### Arreglo de la variable time2:
- División:

In [136]:
time2_norm, scale_factor_time = era5.normalization(time2_parches[:inp_norm.shape[0]])
time2_tr =time2_norm[:inp_tr.shape[0]]
time2_te = time2_norm[inp_tr.shape[0]:inp_tr.shape[0]+inp_va.shape[0]]
time2_va = time2_norm[inp_tr.shape[0]+inp_va.shape[0]:]

In [138]:
time2_tr= tf.gather(time2_tr, indices_tr)
time2_te= tf.gather(time2_te, indices_te)
time2_va= tf.gather(time2_va, indices_va)

#### Revisión dimensiones: 

In [139]:
print("#"*10+" "+"Entrenamiento"+" "+"#"*10)
print("target: ",target_tr.shape)
print("inp: ",inp_tr.shape)
print("lat: ",lat_tr.shape)
print("long: ",lon_tr.shape)
print("orog: ",orog_tr.shape)
print("time1: ",time2_tr.shape)

########## Entrenamiento ##########
target:  (10526, 3, 32, 32, 1)
inp:  (10526, 3, 32, 32, 1)
lat:  (10526, 3, 32, 32, 1)
long:  (10526, 3, 32, 32, 1)
orog:  (10526, 3, 32, 32, 1)
time1:  (10526, 3, 4, 4, 1)


In [140]:
print("#"*10+" "+"Validación"+" "+"#"*10)
print("target: ",target_va.shape)
print("inp: ",inp_va.shape)
print("lat: ",lat_va.shape)
print("long: ",lon_va.shape)
print("orog: ",orog_va.shape)
print("time1: ",time2_va.shape)

########## Validación ##########
target:  (3509, 3, 32, 32, 1)
inp:  (3509, 3, 32, 32, 1)
lat:  (3509, 3, 32, 32, 1)
long:  (3509, 3, 32, 32, 1)
orog:  (3509, 3, 32, 32, 1)
time1:  (3509, 3, 4, 4, 1)


In [142]:
print("#"*10+" "+"Testeo"+" "+"#"*10)
print("target: ",target_te.shape)
print("inp: ",inp_te.shape)
print("lat: ",lat_te.shape)
print("long: ",lon_te.shape)
print("orog: ",orog_te.shape)
print("time1: ",time2_te.shape)

########## Testeo ##########
target:  (3509, 3, 32, 32, 1)
inp:  (3509, 3, 32, 32, 1)
lat:  (3509, 3, 32, 32, 1)
long:  (3509, 3, 32, 32, 1)
orog:  (3509, 3, 32, 32, 1)
time1:  (3509, 3, 4, 4, 1)


In [143]:
import Model as cnn
model_grad_loss = cnn.get_model()

2.11.0


In [144]:
tf.config.run_functions_eagerly(True)
grad_loss=model_grad_loss.fit([inp_tr,time2_tr,lat_tr,lon_tr,orog_tr], target_tr, epochs=2,
            validation_data=([inp_va,time2_va,lat_va,lon_va,orog_va], target_va))

c:\Users\rlagos\anaconda3\envs\sivar\lib\site-packages\tensorflow\python\data\ops\structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/2
134/329 [===========>..................] - ETA: 1:51:39 - loss: 1.8517 - mse: 383.0214 - mae: 1.6262 - mape: 392.5706